In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from mySVM.model import SVM
import itertools 
import multiprocessing as mp
import numpy as np

In [2]:
feature_dir = 'Data/FeatureVectors/iFeatureVectors/'
feature_files = [f.name for f in os.scandir(feature_dir)]

In [3]:
label_dir = 'Data/Labels/'
label_file = 'Labels_Classification.csv'

In [4]:
def model_evaluate(filename,random_seed):
    df1 = pd.read_csv(feature_dir+filename,header=None)
    df2 = pd.read_csv(label_dir+label_file,header=None)
    df = df1.merge(df2,on=0)
    enz_names = df[0].values
    X = df.iloc[:,1:-1].values
    y = df.iloc[:,-1].values
    if X.shape[1]<40:
        n_comp = int(0.75*X.shape[1])
    else:
        n_comp=40
    X_train, X_test, y_train, y_test,enz_train,enz_test = train_test_split(X, y,enz_names, test_size=0.25, random_state=random_seed)
    svm = SVM(X_train,X_test,y_train,y_test,verbose=False,optimize=False, pca_comp=n_comp,kern='rbf',regC=30)
    return svm.acc_train,svm.acc_test
 

In [5]:
mp.cpu_count()

24

In [6]:
pool = mp.Pool(mp.cpu_count())

In [7]:
iterable = list(itertools.product(feature_files,range(1000)))

In [8]:
%%time
model_accs = list(pool.starmap(model_evaluate,iterable))

CPU times: user 498 ms, sys: 171 ms, total: 669 ms
Wall time: 7min 10s


In [9]:
mydict = dict()

for m,acc in zip(iterable,model_accs):
    model_name=m[0].replace('.csv','')
    seed = m[1]
    if model_name not in mydict:
        mydict[model_name] = {seed:acc}
    else:
        mydict[model_name][seed] = acc

In [10]:
def get_model_stats(mname):
    train_accs = [v[0] for k,v in mydict[mname].items()]
    test_accs = [v[1] for k,v in mydict[mname].items()]
    return {'train_min':min(train_accs),'train_mean':np.mean(train_accs),
            'train_max':max(train_accs),'train_std':np.std(train_accs),
            'test_min':min(test_accs),'test_mean':np.mean(test_accs),
            'test_max':max(test_accs),'test_std':np.std(test_accs)}

In [11]:
get_model_stats('Geary')

{'train_min': 0.8275862068965517,
 'train_mean': 0.8992988505747126,
 'train_max': 0.9655172413793104,
 'train_std': 0.02295364886105179,
 'test_min': 0.4482758620689655,
 'test_mean': 0.7134827586206898,
 'test_max': 0.9310344827586207,
 'test_std': 0.08077742794640187}

In [12]:
results_dict= {mname:get_model_stats(mname) for mname in mydict.keys()}
    

In [13]:
results_df = pd.DataFrame(results_dict).T

In [14]:
results_df.to_csv('Data/SimResults/iFeatResults.csv',index=True)

In [15]:
results_df.sort_values('train_mean')

,train_min,train_mean,train_max,train_std,test_min,test_mean,test_max,test_std
GAAC,0.609195,0.706103,0.804598,0.029905,0.275862,0.518931,0.793103,0.080564
NMBroto,0.816092,0.882609,0.965517,0.024651,0.448276,0.688172,0.896552,0.078853
CKSAAP,0.816092,0.888345,0.954023,0.024062,0.448276,0.703138,0.931034,0.076487
SOCNumber,0.827586,0.891149,0.965517,0.024848,0.413793,0.684448,0.931034,0.077308
TPC,0.816092,0.892000,0.954023,0.024473,0.413793,0.696931,0.896552,0.077907
CTDD,0.839080,0.897115,0.954023,0.019297,0.344828,0.637414,0.862069,0.080425
Geary,0.827586,0.899299,0.965517,0.022954,0.448276,0.713483,0.931034,0.080777
Moran,0.839080,0.903149,0.965517,0.022479,0.448276,0.714621,0.931034,0.079489
DPC,0.850575,0.909920,0.977011,0.019850,0.413793,0.712448,0.931034,0.077383
PAAC,0.850575,0.914782,0.977011,0.018270,0.413793,0.640034,0.862069,0.080768
